In [1]:
!pip install -U --no-deps /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -U --no-deps /kaggle/input/datasets-214/datasets-2.14.5-py3-none-any.whl
!pip install -U --no-deps /kaggle/input/optimum-113/optimum-1.13.2-py3-none-any.whl

Processing /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/datasets-214/datasets-2.14.5-py3-none-any.whl
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
Processing /kaggle/input/optimum-113/optimum-1.13.2-py3-none-any.whl


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/saved_model.pb
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/keras_metadata.pb
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/assets/vocab.txt
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/variables/variables.index
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/variables/variables.data-00000-of-00001
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/saved_model.pb
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/keras_metadata.pb
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/assets/vocab.txt
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/variables/variables.index
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/variables/variables.data-00000-of-00001
/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv
/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv
/kaggle/in

**SETTING**

In [3]:
class SentenceTransformer:
    def __init__(self, checkpoint, device="cuda:1"):
        self.device = device
        self.checkpoint = checkpoint
        self.model = AutoModel.from_pretrained(checkpoint).to(self.device).half()
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    def transform(self, batch):
        tokens = self.tokenizer(batch["text"], truncation=True, padding=True, return_tensors="pt", max_length=MAX_SEQ_LEN)
        return tokens.to(self.device)  

    def get_dataloader(self, sentences, batch_size=32):
        sentences = ["Represent this sentence for searching relevant passages: " + x for x in sentences]
        dataset = Dataset.from_dict({"text": sentences})
        dataset.set_transform(self.transform)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        return dataloader

    def encode(self, sentences, show_progress_bar=False, batch_size=1):
        dataloader = self.get_dataloader(sentences, batch_size=batch_size)
        pbar = tqdm(dataloader) if show_progress_bar else dataloader

        embeddings = []
        for batch in pbar:
            with torch.no_grad():
                e = self.model(**batch).pooler_output
                e = F.normalize(e, p=2, dim=1)
                embeddings.append(e.detach().cpu().numpy())
        embeddings = np.concatenate(embeddings, axis=0)
        return embeddings

In [4]:
from time import time
MODEL_PATH = "/kaggle/input/bge-small-faiss/"
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, AutoModel
import gc
# For RAG
import faiss
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_from_disk, Dataset
MAX_SEQ_LEN = 512
import torch
import ctypes
def clean_memory():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()
    
    
NUM_TITLES = 5

In [5]:
import keras
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import random
from sklearn.model_selection import KFold
from tensorflow.keras.layers import concatenate

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
LABEL_DICT = {"A":0,'B':1,'C':2,'D':3,'E':4}

In [7]:
def read_data(df,check_test= False):
    X_Prompt = tf.convert_to_tensor(df.prompt.to_list()) 
    X_Context = tf.convert_to_tensor(df.context.to_list()) 
#     X_Context = tf.convert_to_tensor(df.context.apply(lambda x: x[:10]).to_list()) 
    X_A = tf.convert_to_tensor(df.A.astype(str).to_list())
    X_B = tf.convert_to_tensor(df.B.astype(str).to_list())
    X_C = tf.convert_to_tensor(df.C.astype(str).to_list())
    X_D = tf.convert_to_tensor(df.D.astype(str).to_list())
    X_E = tf.convert_to_tensor(df.E.astype(str).to_list())  
    
    Selects = {'A':X_A,"B":X_B,"C":X_C,'D':X_D,'E':X_E}
    
    Labels = labels_one_hot = None
    
    
    if not check_test:
        labels = df.answer.to_list()
        labels_one_hot = [LABEL_DICT.get(i) for i in labels]

        label_A = []
        label_B = []
        label_C = []
        label_D = []
        label_E = []

        for i in labels:
            if i =='A':
                label_A.append(0.8)
                label_B.append(0.05)
                label_C.append(0.05)
                label_D.append(0.05)  
                label_E.append(0.05)
            if i =='B':
                label_B.append(0.8)
                label_A.append(0.05)
                label_C.append(0.05)
                label_D.append(0.05)  
                label_E.append(0.05)

            if i =='C':
                label_C.append(0.8)
                label_A.append(0.05)
                label_B.append(0.05)
                label_D.append(0.05)  
                label_E.append(0.05)
            if i =='D':
                label_D.append(0.8)
                label_A.append(0.05)
                label_B.append(0.05)
                label_C.append(0.05) 
                label_E.append(0.05)
            if i =='E':
                label_E.append(0.8)
                label_A.append(0.05)
                label_B.append(0.05)
                label_C.append(0.05)
                label_D.append(0.05)  
        label_A = tf.convert_to_tensor(label_A)
        label_B = tf.convert_to_tensor(label_B)
        label_C = tf.convert_to_tensor(label_C)
        label_D = tf.convert_to_tensor(label_D)
        label_E = tf.convert_to_tensor(label_E)


        Labels  = {'A': label_A,'B': label_B,'C': label_C,'D': label_D,'E': label_E}
    return {'Prompt':X_Prompt,'Context': X_Context,'Selects':Selects,'labels':Labels,'label_onehot':labels_one_hot}

**INPUT TRAIN**

*4 thread model*

*Binary model*

In [8]:
model_faiss = SentenceTransformer(MODEL_PATH, device="cuda:1")

In [9]:
path = ['/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv', '/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv']
df = pd.read_csv(path[0])
lst = ['prompt','A','B','C','D','E','answer']
df = df[lst]
for i in path[1:]:
    newdf = pd.read_csv(i)[lst]
    df = pd.concat([df,newdf])
df.reset_index(drop=True, inplace=True)

In [10]:
df_test = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv').drop(columns='id')

In [11]:
df['train'] = True
df_test['train'] = False
df = pd.concat([df,df_test]).reset_index(drop=True)
df.head()

,prompt,A,B,C,D,E,answer,train
0,"In relation to Eunice Fay McKenzie's career, w...",McKenzie showcased her singing talents in nume...,McKenzie is primarily remembered for her starr...,McKenzie gained recognition for her role as a ...,McKenzie's collaborations with director Blake ...,McKenzie's successful career in sound films co...,B,True
1,How does Modified Newtonian Dynamics (MOND) im...,MOND is a theory that increases the discrepanc...,MOND explains the missing baryonic mass in gal...,MOND is a theory that reduces the observed mis...,MOND is a theory that eliminates the observed ...,MOND's impact on the observed missing baryonic...,E,True
2,Which of the following statements accurately d...,Ray Montgomerie is a former footballer who pla...,Ray Montgomerie is a former footballer who pla...,Ray Montgomerie is a former footballer who pla...,Ray Montgomerie is a former footballer who pla...,Ray Montgomerie is a former footballer who pla...,B,True
3,What is the significance of the Museum of the ...,The Museum of the Occupation of Latvia is a me...,The Museum of the Occupation of Latvia showcas...,The Museum of the Occupation of Latvia was est...,The Museum of the Occupation of Latvia primari...,The Museum of the Occupation of Latvia is a mu...,C,True
4,What was the previous name of the Christian Sc...,The Christian School for the Deaf (CSD),The Christian School for the Blind (CSB),The Evangelical School and Chapel for the Deaf...,The Evangelical School for the Deaf (ESD),The Evangelical School for the Blind (ESB),D,True


In [12]:
# Load embedding model
start = time()
print(f"Starting prompt embedding, t={time() - start :.1f}s")

# Get embeddings of prompts
f = lambda row : " ".join([str(row["prompt"]), str(row["A"]), str(row["B"]), str(row["C"]), str(row["D"]), str(row["E"])])
inputs = df.apply(f, axis=1).values # better results than prompt only
prompt_embeddings = model_faiss.encode(inputs, show_progress_bar=False)

# Search closest sentences in the wikipedia index 
print(f"Loading faiss index, t={time() - start :.1f}s")
faiss_index = faiss.read_index(MODEL_PATH + '/faiss.index')
# faiss_index = faiss.index_cpu_to_all_gpus(faiss_index) # causes OOM, and not that long on CPU

print(f"Starting text search, t={time() - start :.1f}s")
search_index = faiss_index.search(np.float32(prompt_embeddings), NUM_TITLES)[1]

print(f"Starting context extraction, t={time() - start :.1f}s")
dataset = load_from_disk("/kaggle/input/all-paraphs-parsed-expanded")
for i in range(len(df)):
    df.loc[i, "context"] = "-" + "\n-".join([dataset[int(j)]["text"] for j in search_index[i]])

# Free memory
faiss_index.reset()
del faiss_index, prompt_embeddings, dataset
clean_memory()
print(f"Context added, t={time() - start :.1f}s")

Starting prompt embedding, t=0.0s
Loading faiss index, t=94.0s
Starting text search, t=122.0s
Starting context extraction, t=173.7s
Context added, t=197.6s


In [13]:
result = read_data(df[df['train']],False)
X_Prompt = result.get('Prompt') 
X_Context = result.get('Context') 
X_A = result.get('Selects').get('A')
X_B = result.get('Selects').get('B')
X_C = result.get('Selects').get('C')
X_D = result.get('Selects').get('D')
X_E = result.get('Selects').get('E')

In [14]:
Selects = [result.get('Selects')[i] for i in result.get('Selects') ]
Selects.insert(0,X_Context)
Selects.insert(0,X_Prompt)
labels = [result.get('labels')[i] for i in result.get('labels') ]
labels_one_hot = result.get('label_onehot')
labels_one_hot_train = tf.one_hot(labels_one_hot, depth=5)

In [15]:
result_val = read_data(df[~df['train']] ,False)
X_Prompt_val  = result_val.get('Prompt') 
X_Context_val = result_val.get('Context') 
X_A_val  = result_val .get('Selects').get('A')
X_B_val  = result_val .get('Selects').get('B')
X_C_val  = result_val .get('Selects').get('C')
X_D_val  = result_val .get('Selects').get('D')
X_E_val  = result_val .get('Selects').get('E')

Selects_val  = [result_val .get('Selects')[i] for i in result_val .get('Selects') ]
Selects_val .insert(0,X_Context_val )
Selects_val .insert(0,X_Prompt_val )
labels_val  = [result_val .get('labels')[i] for i in result_val .get('labels') ]
labels_one = result_val.get('label_onehot')
labels_one_hot_val = tf.one_hot(labels_one, depth=5)

**LOAD MODEL BERT**

In [16]:
bert_preprocess = hub.KerasLayer("/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3")
bert_encoder = hub.KerasLayer("/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2")

**BUILD MODEL**

In [17]:
text_Prompt = tf.keras.layers.Input(shape=(), dtype=tf.string)
text_Context = tf.keras.layers.Input(shape=(), dtype=tf.string)
text_A = tf.keras.layers.Input(shape=(), dtype=tf.string)
text_B = tf.keras.layers.Input(shape=(), dtype=tf.string)
text_C = tf.keras.layers.Input(shape=(), dtype=tf.string)
text_D = tf.keras.layers.Input(shape=(), dtype=tf.string)
text_E = tf.keras.layers.Input(shape=(), dtype=tf.string)

encoder_prompt_input = bert_preprocess(text_Prompt)
encoder_context_input = bert_preprocess(text_Context)
encoder_A_input = bert_preprocess(text_A)
encoder_B_input = bert_preprocess(text_B)
encoder_C_input = bert_preprocess(text_C)
encoder_D_input = bert_preprocess(text_D)
encoder_E_input = bert_preprocess(text_E)

In [18]:
encoder_prompt = concatenate(
        tuple([bert_encoder(encoder_prompt_input)['encoder_outputs'][i] for i in range(-4, 0)]), 
        name = 'hidden_states_prompt_1',
        axis = -1
    )[:, 0, :]
encoder_context = concatenate(
        tuple([bert_encoder(encoder_context_input)['encoder_outputs'][i] for i in range(-4, 0)]), 
        name = 'hidden_states_context_1',
        axis = -1
    )[:, 0, :]
encoder_A = concatenate(
        tuple([bert_encoder(encoder_A_input)['encoder_outputs'][i] for i in range(-4, 0)]), 
        name = 'hidden_states_A',
        axis = -1
    )[:, 0, :]
encoder_B = concatenate(
        tuple([bert_encoder(encoder_B_input)['encoder_outputs'][i] for i in range(-4, 0)]), 
        name = 'hidden_states_B',
        axis = -1
    )[:, 0, :]
encoder_C = concatenate(
        tuple([bert_encoder(encoder_C_input)['encoder_outputs'][i] for i in range(-4, 0)]), 
        name = 'hidden_states_C',
        axis = -1
    )[:, 0, :]
encoder_D = concatenate(
        tuple([bert_encoder(encoder_D_input)['encoder_outputs'][i] for i in range(-4, 0)]), 
        name = 'hidden_states_D',
        axis = -1
    )[:, 0, :]
encoder_E = concatenate(
        tuple([bert_encoder(encoder_E_input)['encoder_outputs'][i] for i in range(-4, 0)]), 
        name = 'hidden_states_E',
        axis = -1
    )[:, 0, :]
print(encoder_E.shape)

reshape_prompt = tf.reshape(encoder_prompt, (-1, 4, 768))
reshape_context = tf.reshape(encoder_context, (-1, 4, 768))
reshape_A = tf.reshape(encoder_A, (-1, 4, 768))
reshape_B = tf.reshape(encoder_B, (-1, 4, 768))
reshape_C = tf.reshape(encoder_C, (-1, 4, 768))
reshape_D = tf.reshape(encoder_D, (-1, 4, 768))
reshape_E = tf.reshape(encoder_E, (-1, 4, 768))
print('reshape: ', reshape_A.shape)
# layer A
concat_encoder_A = concatenate([reshape_prompt,reshape_context, reshape_A,reshape_B,reshape_C,reshape_D,reshape_E],axis = 1)
LSTM = tf.keras.layers.LSTM(units=32,dropout=0.2, recurrent_dropout=0.2,return_sequences=True)(concat_encoder_A)
Dense_prompt_A = tf.keras.layers.Dense(32,activation = 'relu')(LSTM)
Dense_prompt_A =  tf.keras.layers.Dropout(0.3)(Dense_prompt_A)
Dense_prompt_A = tf.keras.layers.LayerNormalization()(Dense_prompt_A)
Linear_A =  tf.keras.layers.Dropout(0.3)(Dense_prompt_A)
classifer = keras.layers.Flatten()(Linear_A)
classifer = tf.keras.layers.Dense(5,activation = 'linear')(classifer)
classifer =  tf.keras.layers.Dense(5,activation = 'softmax')(classifer)

model = keras.Model(inputs=[text_Prompt, text_Context, text_A,text_B,text_C,text_D,text_E], outputs=classifer)
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.summary()

(None, 3072)
reshape:  (None, 4, 768)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None,)]                    0         []                            
                                                        

In [19]:
EarlyStopping = tf.keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    patience=10,
                    restore_best_weights=True,
                )

In [20]:
# tf.config.run_functions_eagerly(True)

In [21]:
model.fit(x = Selects,y = labels_one_hot_train,batch_size=32,
          epochs=50,  steps_per_epoch=60,
         validation_data=(Selects_val, labels_one_hot_val) , callbacks=[EarlyStopping])# 50 epoch

Epoch 1/50
60/60 [==============================] - 560s 8s/step - loss: 2.1730 - accuracy: 0.1885 - val_loss: 1.6369 - val_accuracy: 0.2400
Epoch 2/50
60/60 [==============================] - 500s 8s/step - loss: 1.8596 - accuracy: 0.2135 - val_loss: 1.6509 - val_accuracy: 0.2350
Epoch 3/50
60/60 [==============================] - 500s 8s/step - loss: 1.7702 - accuracy: 0.2000 - val_loss: 1.6386 - val_accuracy: 0.1700
Epoch 4/50
60/60 [==============================] - 493s 8s/step - loss: 1.7065 - accuracy: 0.2019 - val_loss: 1.6147 - val_accuracy: 0.1950
Epoch 5/50
60/60 [==============================] - 500s 8s/step - loss: 1.6708 - accuracy: 0.2068 - val_loss: 1.6251 - val_accuracy: 0.2150
Epoch 6/50
60/60 [==============================] - 500s 8s/step - loss: 1.6565 - accuracy: 0.2104 - val_loss: 1.6229 - val_accuracy: 0.1850
Epoch 7/50
60/60 [==============================] - 492s 8s/step - loss: 1.6478 - accuracy: 0.2019 - val_loss: 1.6258 - val_accuracy: 0.1900
Epoch 8/50
60

In [22]:
# model.save('19_epoch_13.h5')

In [23]:
# y_predict = model.predict(Selects)
# top_three_indices = (-y_predict).argsort(axis = 1)[:, :3].tolist()
# top_max = np.argmax(y_predict,axis = -1)
# report = classification_report(labels_one_hot, top_max)
# print(report)

In [24]:
# count = 0
# for i in range(len(labels_one_hot)):
#     if labels_one_hot[i] in top_three_indices[i]:
#         count +=1
# print(count)

In [25]:
y_predict = model.predict(Selects_val)
top_three_indices = (-y_predict).argsort(axis = 1)[:, :3].tolist()
top_max = np.argmax(y_predict,axis = -1)
report = classification_report(labels_one, top_max)
print(report)

7/7 [==============================] - 65s 7s/step
              precision    recall  f1-score   support

           0       0.52      0.30      0.38        37
           1       0.31      0.35      0.33        48
           2       0.41      0.45      0.43        44
           3       0.43      0.26      0.33        38
           4       0.40      0.64      0.49        33

    accuracy                           0.40       200
   macro avg       0.42      0.40      0.39       200
weighted avg       0.41      0.40      0.39       200



In [26]:
count = 0
for i in range(len(labels_one)):
    if labels_one[i] in top_three_indices[i]:
        count +=1
print(count)

158


**DATA TEST**

In [27]:
path_test = ['/kaggle/input/kaggle-llm-science-exam/test.csv']
df_scoring = pd.read_csv(path_test[0])

In [28]:
# Load embedding model
start = time()
print(f"Starting prompt embedding, t={time() - start :.1f}s")

# Get embeddings of prompts
f = lambda row : " ".join([str(row["prompt"]), str(row["A"]), str(row["B"]), str(row["C"]), str(row["D"]), str(row["E"])])
inputs = df_scoring.apply(f, axis=1).values # better results than prompt only
prompt_embeddings = model_faiss.encode(inputs, show_progress_bar=False)

# Search closest sentences in the wikipedia index 
print(f"Loading faiss index, t={time() - start :.1f}s")
faiss_index = faiss.read_index(MODEL_PATH + '/faiss.index')
# faiss_index = faiss.index_cpu_to_all_gpus(faiss_index) # causes OOM, and not that long on CPU

print(f"Starting text search, t={time() - start :.1f}s")
search_index = faiss_index.search(np.float32(prompt_embeddings), NUM_TITLES)[1]

print(f"Starting context extraction, t={time() - start :.1f}s")
dataset = load_from_disk("/kaggle/input/all-paraphs-parsed-expanded")
for i in range(len(df_scoring)):
    df_scoring.loc[i, "context"] = "-" + "\n-".join([dataset[int(j)]["text"] for j in search_index[i]])

# Free memory
faiss_index.reset()
del faiss_index, prompt_embeddings, dataset
clean_memory()
print(f"Context added, t={time() - start :.1f}s")

Starting prompt embedding, t=0.0s
Loading faiss index, t=4.2s
Starting text search, t=7.1s
Starting context extraction, t=9.9s
Context added, t=12.2s


In [29]:
result = read_data(df_scoring,True)
X_Prompt = result.get('Prompt') 
X_Context = result.get('Context') 
X_A = result.get('Selects').get('A')
X_B = result.get('Selects').get('B')
X_C = result.get('Selects').get('C')
X_D = result.get('Selects').get('D')
X_E = result.get('Selects').get('E')
Selects = [result.get('Selects')[i] for i in result.get('Selects') ]
Selects.insert(0,X_Context)
Selects.insert(0,X_Prompt)

In [30]:
y_predict = model.predict(Selects)
top_three_indices = (-y_predict).argsort(axis = 1)[:, :3].tolist()

7/7 [==============================] - 46s 6s/step


In [31]:
# Mở tệp CSV để ghi
df_resutl =  pd.read_csv("/kaggle/input/kaggle-llm-science-exam/sample_submission.csv")
x = ['A','B','C','D',"E"]
Label_test = [' '.join([x[int(j)] for j in i]) for i in top_three_indices]

for i in range(len(Label_test)):
    df_resutl.at[i, 'prediction'] = str(Label_test[i])
df_resutl.to_csv('/kaggle/working/submission.csv', index=False)